In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# make local editable packages automatically reload
# Import dependencies
import numpy as np
from cellpose_omni import core
# This checks to see if you have set up your GPU properly.
# CPU performance is a lot slower, but not a problem if you 
# are only processing a few images.
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)
# for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 300
plt.style.use('dark_background')
from natsort import natsorted
from glob import glob
from PIL import Image
from cellpose_omni import transforms
from omnipose.utils import normalize99
from cellpose_omni import models
from joblib import Parallel, delayed
from tqdm.auto import tqdm
%matplotlib inline

2023-05-10 03:28:29,884 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:29,886 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:30,375 [INFO] ** TORCH GPU version installed and working. **
>>> GPU activated? 1


In [2]:
data_dirs = glob("data/*")

In [3]:
data_dirs += ["pretrained"]

In [4]:
def seg_data(data_dir):
#for data_dir in data_dirs:
    imgs = []
    image_dirs = natsorted(glob(f"{data_dir}/test_data/*"))
    non_mask_dirs = []
    for image_dir in image_dirs:
        if "mask" not in image_dir:
            imgs.append(
                np.array(Image.open(image_dir))
            )
            non_mask_dirs.append(image_dir)

            

    nimg = len(imgs)

    for k in range(len(imgs)):
        img = transforms.move_min_dim(imgs[k]) # move the channel dimension last
        if len(img.shape)>2:
            # imgs[k] = img[:,:,1] # could pick out a specific channel
            imgs[k] = np.mean(img,axis=-1) # or just turn into grayscale 

        imgs[k] = normalize99(imgs[k])
        
    try:
        os.mkdir(f"{data_dir}/Omnipose_segmented_by")
    except:
        pass
    
    
    for model_data_dir in data_dirs:
        if model_data_dir == "pretrained":
            
            model_list = natsorted(glob(f"{model_data_dir}/training_data/models/*"))
            model_name = "bact_fluor_omni"
            use_gpu = use_GPU# = False
            model = models.CellposeModel(gpu=use_gpu, model_type=model_name, omni=True, concatenation=True)
            seg_by = "bact_fluor_omni"
        
        else:
            model_list = natsorted(glob(f"{model_data_dir}/training_data/models/*"))
            model_name = model_list[-1]
            use_gpu = use_GPU# = False
            model = models.CellposeModel(gpu=use_gpu, pretrained_model=model_name, omni=True, concatenation=True)
            seg_by = model_data_dir.split("/")[1]
            
        try:
            os.mkdir(f"{data_dir}/Omnipose_segmented_by/{seg_by}")
        except:
            pass
        
        chans = [0,0] #this means segment based on first channel, no second channel

        n = [0] # make a list of integers to select which images you want to segment
        n = range(nimg) # or just segment them all

        # define parameters
        mask_threshold = -1
        verbose = 0 # turn on if you want to see more output
        use_gpu = use_GPU #defined above
        transparency = True # transparency in flow output
        rescale=None # give this a number if you need to upscale or downscale your images
        omni = True # we can turn off Omnipose mask reconstruction, not advised
        flow_threshold = 0. # default is .4, but only needed if there are spurious masks to clean up; slows down output
        resample = True #whether or not to run dynamics on rescaled grid or original grid
        masks, flows, styles = model.eval([imgs[i] for i in n],channels=chans,rescale=rescale,mask_threshold=mask_threshold,transparency=transparency,
                                          flow_threshold=flow_threshold,omni=omni,resample=resample,verbose=verbose)

        for mask, savename in zip(masks, non_mask_dirs):
            filename = savename.split("/")[-1]
            Image.fromarray(mask).save(f"{data_dir}/Omnipose_segmented_by/{seg_by}/{filename}")
            
_ = Parallel(n_jobs=-1)(delayed(seg_data)(data_dir) for data_dir in tqdm(data_dirs))

  0%|          | 0/7 [00:00<?, ?it/s]

2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,085 [INFO] OpenGL_accelerate module loaded
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,087 [INFO] Using accelerated ArrayDatatype
2023-05-10 03:28:35,547 [INFO] ** TORCH GPU version installed and working. **
2023-05-10 03:28:35,547 [INFO] >>>> usin